# WRFcast Tutorial

This tutorial will walk through forecast data from your own WRF forecast model data using the wrfcast.py module within pvlib.

This tutorial has been tested against the following package versions:
* Python 
* IPython 
* pandas 
* matplotlib 
* netcdf4 1.4.2 

It should work with other Python and Pandas versions. It requires pvlib >= 0.3.0 and IPython >= 3.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, November 2019

In [120]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf
import pvlib
from pvlib.wrfcast import WRF

In [121]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
netcdf_data = netCDF4.Dataset(datapath)


In [133]:
netcdf_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE: 2011-01-24_00:00:00
    SIMULATION_START_DATE: 2011-01-23_12:00:00
    WEST-EAST_GRID_DIMENSION: 192
    SOUTH-NORTH_GRID_DIMENSION: 192
    BOTTOM-TOP_GRID_DIMENSION: 36
    DX: 12000.0
    DY: 12000.0
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 10
    RA_LW_PHYSICS: 1
    RA_SW_PHYSICS: 1
    SF_SFCLAY_PHYSICS: 2
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 5
    CU_PHYSICS: 3
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 1
    GFDDA_INTERVAL_M: 180
    GFDDA_END_H: 132
    GRID_SFDDA: 1
    SGFDDA_INTERVAL_M: 360
    SGFDDA_END_H: 120
    HYPSOMETRIC_OPT: 2
    USE_THETA_M: 0
    SF_URBAN_PHYSICS: 0
    SHCU_PHYSICS: 0
    MFSHCONV: 0
    FEEDBACK: 1


In [122]:
# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
            'Times',
            'T2',
            'U10',
            'V10',
            'CLDFRA',
            'SWDDNI',
            'SWDDIF'
            ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key, timeidx=wrf.ALL_TIMES)
    if first:
        solar_data = var
        first = False
    else:
        solar_data = xr.merge([solar_data, var])

variables = {
            'times': 'times',
            'XLAT': 'lat',
            'XLONG': 'lon',
            'T2': 'temp_air',
            'U10': 'wind_speed_u',
            'V10': 'wind_speed_v',
            'CLDFRA': 'total_clouds',
            'SWDDNI': 'dni',
            'SWDDIF': 'dhi' 
            }
solar_data = xr.Dataset.rename(solar_data, variables)
times = solar_data.times
ntimes = solar_data.sizes['Time']
nlat = solar_data.sizes['south_north']
nlon = solar_data.sizes['west_east']
solar_data

<xarray.Dataset>
Dimensions:       (Time: 24, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time          (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-25
    lon           (south_north, west_east) float32 -95.43335 ... -64.11832
    lat           (south_north, west_east) float32 30.952759 ... 47.507103
    XTIME         (Time) float32 780.0 840.0 900.0 ... 2040.0 2100.0 2160.0
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    times         (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-25
    temp_air      (Time, south_north, west_east) float32 287.62427 ... 264.03226
    wind_speed_u  (Time, south_north, west_east) float32 0.31499797 ... 17.972435
    wind_speed_v  (Time, south_north, west_east) float32 1.1208413 ... -1.0850167
    total_clouds  (Time, bottom_top, south_north, west_east) float32 0.0 ... 0.0
    dni           (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0
    dhi           (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0

In [123]:
# Explore how the WRF forecast model behaves
fm = WRF()
wind_speed = fm.uv_to_speed(solar_data)
temp_air = fm.kelvin_to_celsius(solar_data['temp_air'])
# ghi = fm.dni_and_dhi_to_ghi(solar_data['dni'], solar_data['dhi'])

In [124]:
# Convert xarray Datasets to a pandas DataFrames
solar_data = solar_data.to_dataframe()
times = times.to_dataframe()
solar_data

times  \
Time                bottom_top south_north west_east                       
2011-01-24 01:00:00 0          0           0         2011-01-24 01:00:00   
                                           1         2011-01-24 01:00:00   
                                           2         2011-01-24 01:00:00   
                                           3         2011-01-24 01:00:00   
                                           4         2011-01-24 01:00:00   
...                                                                  ...   
2011-01-25 00:00:00 34         190         186       2011-01-25 00:00:00   
                                           187       2011-01-25 00:00:00   
                                           188       2011-01-25 00:00:00   
                                           189       2011-01-25 00:00:00   
                                           190       2011-01-25 00:00:00   

                                                            lon        lat  \
Time                bottom_top south_north west_east                         
2011-01-24 01:00:00 0          0           0         -95.433350  30.952759   
                                           1         -95.308044  30.950832   
                                           2         -95.182739  30.948757   
                                           3         -95.057465  30.946545   
                                           4         -94.932190  30.944153   
...                                                         ...        ...   
2011-01-25 00:00:00 34         190         186       -64.714050  47.657688   
                                           187       -64.564850  47.620308   
                                           188       -64.415833  47.582741   
                                           189       -64.266968  47.545010   
                                           190       -64.118317  47.507103   

                                                       XTIME    temp_air  \
Time                bottom_top south_north west_east                       
2011-01-24 01:00:00 0          0           0           780.0  287.624268   
                                           1           780.0  287.592499   
                                           2           780.0  287.163452   
                                           3           780.0  287.356171   
                                           4           780.0  287.152344   
...                                                      ...         ...   
2011-01-25 00:00:00 34         190         186        2160.0  263.186615   
                                           187        2160.0  262.584229   
                                           188        2160.0  263.205566   
                                           189        2160.0  263.703674   
                                           190        2160.0  264.032257   

                                                      wind_speed_u  \
Time                bottom_top south_north west_east                 
2011-01-24 01:00:00 0          0           0              0.314998   
                                           1             -0.087709   
                                           2             -0.358935   
                                           3             -0.632014   
                                           4             -0.803844   
...                                                            ...   
2011-01-25 00:00:00 34         190         186           15.262465   
                                           187           15.590679   
                                           188           16.902138   
                                           189           17.802485   
                                           190           17.972435   

                                                      wind_speed_v  \
Time                bottom_top south_north west_east                 
2011-01-24 01:00:00 0  

In [135]:
# Run the solar position algorithm for time, lat, and lon indices, and concatonate them into a single DataFrame
numerical_time_indices = range(0, 1)
lat_indices = range(0, 2)
lon_indices = range(0, 2)
first = True
for num_time_idx in numerical_time_indices:
    for lat_idx in lat_indices:
        for lon_idx in lon_indices:
            time = times.index.get_level_values('Time')[num_time_idx]
            print(f'Time Index: {time}')
            print(f'\tLatitude index: {lat_idx}')
            print(f'\t\tLongitude index: {lon_idx}')
            
            solpos_new = pvlib.solarposition.spa_xarray_python(time, 
                                        solar_data['lat'].loc[time, 0, lat_idx, lon_idx], 
                                        solar_data['lon'].loc[time, 0, lat_idx, lon_idx], lat_idx, lon_idx)
            
            if first:
                solpos_new = solpos
                first = False
            else:
                solpos = pd.concat([solpos, solpos_new])
            

Time Index: 2011-01-24 01:00:00
	Latitude index: 0
		Longitude index: 0
Time Index: 2011-01-24 01:00:00
	Latitude index: 0
		Longitude index: 1
Time Index: 2011-01-24 01:00:00
	Latitude index: 1
		Longitude index: 0
Time Index: 2011-01-24 01:00:00
	Latitude index: 1
		Longitude index: 1


In [136]:
solpos

apparent_zenith      zenith  \
Time                south_north west_east                                
2011-01-24 01:00:00 0           0               105.261912  105.261912   
                                1               105.365882  105.365882   
                    1           0               105.289079  105.289079   
                                1               105.393098  105.393098   

                                           apparent_elevation  elevation  \
Time                south_north west_east                                  
2011-01-24 01:00:00 0           0                  -15.261912 -15.261912   
                                1                  -15.365882 -15.365882   
                    1           0                  -15.289079 -15.289079   
                                1                  -15.393098 -15.393098   

                                              azimuth  equation_of_time  
Time                south_north west_east                                
2011-01-24 01:00:00 0           0          256.321518        -11.883784  
                                1          256.378509        -11.883784  
                    1           0          256.351019        -11.883784  
                                1          256.408303        -11.883784